In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *
from packages.restaurant.mulProcess_restaurant_scraping_proxy import *

import os
import glob
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values 

from multiprocessing import Pool
from functools import partial

In [2]:
# *** following this step to use multiprocessing method for restaurant scraping with remote scraping browser
# - select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
# - find out which is the last page of a province in wongnai.com (manually)

# ** select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
Idx_of_region = 63
cur_region_data = ALL_REGION_ID_WONGNAI[Idx_of_region]

# ** find out which is the last page of a province in wongnai.com (manually)
last_page_number = 281

#
cur_province_en = cur_region_data[0]
cur_province_th = cur_region_data[1]
cur_regionId = cur_region_data[2]

# create list of : [1, 2, 3, ...., last_page]
# page_number_list = np.arange(last_page_number) + 1
page_number_list = np.arange(5) + 1
print("check page number list: ")
print(page_number_list)

# get list of dataframe result of all restaurants in a province
# res_allRestaurants_list = []
with Pool(processes=8) as pool: 
    print("a 1")
    func_param_fix = partial(mulProcess_helper_scrape_restaurants_by_province, province = cur_province_th, wongnai_regionId = cur_regionId)
    print("a 2")
    res_allRestaurants_list = pool.map(func_param_fix, page_number_list)
    print("end process for province --> %s " % (cur_province_th))

# concat all dataframe into a final one to save in csv.
res_allRestaurants_df = pd.DataFrame()
print("check len of list result dataframe --> ", len(res_allRestaurants_list))
for cur_df in res_allRestaurants_list:
    print("cur_df :")
    print(cur_df)
    res_allRestaurants_df = pd.concat([res_allRestaurants_df, cur_df])
    print("res_allRestaurants_df :")
    print(res_allRestaurants_df)

print("last check res :")
print(res_allRestaurants_df)
# remove duplicate restaurant 
res_allRestaurants_df.drop_duplicates(subset=['name', 'sub_name'], inplace=True)
# set new index
res_allRestaurants_df.set_index(['name', 'sub_name'], inplace=True)

# save result dataframe to .csv
res_file_name = 'res_restaurant_%s.csv' % (cur_province_en)
res_path = os.path.join(fh.STORE_RESTAURANT_SCRAPING, 'res_restaurant_scraping', res_file_name) 
res_allRestaurants_df.to_csv(res_path, encoding="utf-8")
    

check page number list: 
[1 2 3 4 5]
a 1
a 2
end process for province --> มุกดาหาร 
check len of list result dataframe -->  5
cur_df :
                                                name   sub_name  \
0                                          Pizza Hut   มุกดาหาร   
0                                             มหาศาล     มหาศาล   
0                          The Diary Cafe and Bistro          -   
0                                บ้านลาวญวน ณ ริมโขง              
0                                      ศิริชัยหมูหัน  แก่งกะเบา   
0                    ดา ข้าวต้มปลา (ชลบุรี) มุกดาหาร              
0                                     วีที แหนมเนือง   มุกดาหาร   
0  โรตีบังแอ๊ด ชาชัก ,ครัวบังแอ๊ด (ข้างที่ว่าการอ...              
0                                  วิรินทร์ แจ่วฮ้อน              
0                   ขนมถ้วย ขนมเหนียว หน้าสถานีตำรวจ              

                                         wongnai_url  \
0  https://www.wongnai.com/restaurants/1658993FW-...   
0  https://www.